In [1]:
!pip install openai opencv-python numpy

  Using cached opencv_python-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
  Using cached numpy-2.1.3-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached anyio-4.6.2.post1-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.8.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached pydantic-2.10.2-py3-none-any.whl.metadata (170 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.27.1

In [ ]:
import time
import cv2
import base64
import numpy as np
from openai import OpenAI
import openai.pydantic_function_tool as pft

client = OpenAI(
    api_key=''
)

In [ ]:
cam = cv2.VideoCapture(0)

# Get the default frame width and height
frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (frame_width, frame_height))

while True:
    ret, frame = cam.read()

    # Write the frame to the output file
    out.write(frame)

    # Display the captured frame
    cv2.imshow('Camera', frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) == ord('q'):
        break

    if not ret:
        break
    
    # Encode the current frame as a base64 string
    _, buffer = cv2.imencode('.jpg', frame)
    image_base64 = base64.b64encode(buffer).decode('utf-8')

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Is the fist open or closed?",
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url":  f"data:image/jpeg;base64,{image_base64}"
                        },
                    },
                ],
            }
        ],
    )

    print(response.choices[0].message.content)
    if "open" in response.choices[0].message.content:
        print("Move forward!")
    elif "closed" in response.choices[0].message.content:
        print("Stop!")
    time.sleep(1)

# Release the capture and writer objects
cam.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# specify actions as tools

class Move(BaseModel):
  steps: int

class Turn(BaseModel):
  rotation: int

class Bark(BaseModel):
  pass

tools = [pft(Move), pft(Turn), pft(Bark)]

In [ ]:
def get_audio():
    return 'This is a test audio text'

def get_image():
    ret, frame = cam.read()

    # Write the frame to the output file
    out.write(frame)

    # Display the captured frame
    cv2.imshow('Camera', frame)

    # Encode the current frame as a base64 string
    _, buffer = cv2.imencode('.jpg', frame)
    image_base64 = base64.b64encode(buffer).decode('utf-8')

    return image_base64

def pupper_action_step(prev_summary, audio_text=get_audio(), image=get_image(), ):

    # audio_text: str
    # image: base64 string
    # prev_summary: str

    messages = []
    messages.append({
        "role": "system", 
        "content": "You are a helpful robot assistant. Use the supplied tools to assist the user."
    })
    messages.append({
        "type": "image_url",
        "image_url": {
            "url":  f"data:image/jpeg;base64,{image_base64}"
        },
    })
    messages.append({
        "type": "image_url",
        "image_url": {
            "url":  f"data:image/jpeg;base64,{image_base64}"
        },
    })

    response = client.chat.completions.create(
        model='gpt-4o-2024-08-06',
        messages=messages,
        tools=tools
    )
    response.choices[0].message.tool_calls[0].function
    

    pass


summary = ''

for i in range(20):

    audio_text = get_audio_text()
    image = get_image()

    summary = pupper_action_step(audio_text, image_base64, summary)


response = client.chat.completions.create(
        model="gpt-4o-mini",x
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Is the fist open or closed?",
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url":  f"data:image/jpeg;base64,{image_base64}"
                        },
                    },
                ],
            }
        ],
    )

messages = []
messages.append({
    "role": "system", 
    "content": "You are a helpful robot assistant. Use the supplied tools to assist the user."
})
messages.append({
    "type": "image_url",
    "image_url": {
        "url":  f"data:image/jpeg;base64,{image_base64}"
    },
})
messages.append({
    "type": "image_url",
    "image_url": {
        "url":  f"data:image/jpeg;base64,{image_base64}"
    },
})

response = client.chat.completions.create(
    model='gpt-4o-2024-08-06',
    messages=messages,
    tools=tools
)
response.choices[0].message.tool_calls[0].function